In [1]:
from typing import TypedDict, NamedTuple
from __future__ import annotations

In [2]:
from atptour import *

In [ ]:
driver.find_element(By.XPATH, "//button[text()='2D']").click()


In [ ]:
stroke_dropdown = driver.find_element(By.XPATH, "//div[contains(@class, 'DropdownFixWidth')]")
stroke_dropdown.click()

In [ ]:
options = stroke_dropdown.find_elements(By.XPATH, "./div[@id='RGDropDown']/div[@class='dropdown-container']/div[@class='sublink-container']/*")

In [ ]:
for option in options:
    print (option.text)
    
    
option.click()

In [3]:
def to_camel_case(_s: str, /) -> str:
    # Split the string into words
    words = _s.split()
    
    # Convert the first word to lowercase and keep the rest of the words capitalized
    camel_case = words[0].lower() + ''.join(word.capitalize() for word in words[1:])
    
    return camel_case

def sibling_of_div_text(s: str, /):
    s = s.replace('*', '')
    return driver.find_element(By.XPATH, rf"//div[contains(normalize-space(text()), '{s}')]/following-sibling::div[@class]")

def close_pop_ups():
    active_element = driver.switch_to.active_element
    active_element.send_keys(Keys.ESCAPE)

In [4]:
class Dictable:
    @classmethod
    def to_dict(cls):
        return {k: v for k, v in cls.__dict__.items() if not k.startswith('_')}
    
class Court(Dictable):
    heightM: float = 8.23
    widthM: float = 23.77
    
    
CourtType = type[Court] | Court

In [6]:
from typing import NotRequired  
from typing import Literal


class CourtVision(TypedDict):
    playerSets: list[PlayerSet]


class PlayerSet(TypedDict):
    player: str
    player_index: Literal[1, 2]
    set: Literal[1, 2]
    events: list[Shots]
    
class Shots(TypedDict):
    label: str
    shots: list[Shot]
    
  
class Shot(TypedDict):
    shot_description: str
    type: str
    player: str

    balls: list[Ball]
    
    currentScore: CurrentScore
    attributes: dict[str, str]


class Ball(TypedDict):
    type: str
    x: float
    y: float
    rotate: NotRequired[float]
    

class CurrentScore(TypedDict):
    player1: Score
    player2: Score
    
    
class Score(TypedDict):
    game_score: int
    set_score_1: int
    set_score_2: int | None

In [9]:

import re

def to_real_pos(x: float, y: float, width: float, height: float, court: CourtType):
    x_real = (x / width + 0.5) * court.widthM
    y_real = (0.5 - y / height) * court.heightM 
    
    return x_real, y_real

def extract_first_number(string: str, /):
    numbers = re.findall(r'-?\d+\.\d+|-?\d+', string)
    # Convert the first found number to a float and return it
    if numbers:
        return float(numbers[0])
    return None


def get_selected_group_elements():
    return driver.find_element(By.ID, "plottedBallsSelected")

def get_balls(passed_balls: set[str], court: CourtType) -> list[Ball]:
    height, width = driver.find_element(By.XPATH, '//*[@id="CourtDoublesAlley"]').size.values()
    selected = get_selected_group_elements()
    
    selected_balls = selected.find_elements(By.XPATH, r".//*[local-name()='g']")
    balls: list[Ball] = []
    for ballEl in selected_balls:
        passed_balls.add(ballEl.get_attribute('class'))
        
        
        components = ballEl.find_elements(By.XPATH, r'./*')
        angle = None
        while len(components) > 1:
            for component in components:
                if component.get_attribute('transform') is not None:
                    angle = extract_first_number(component.get_attribute('transform'))
                    components.remove(component)
        if len(components) == 1:
            component = components[0]
            xlinkhref = component.get_attribute('xlink:href')
            typ = xlinkhref.replace('#', '').replace('Selected', '')
            
            x, y = component.get_attribute('x'), component.get_attribute('y')
            x, y = to_real_pos(float(x), float(y), width, height, court)
            balls.append(
                {'x': x,
                'y': y,
                'type': typ}
                )
            
            if angle:
                balls[-1]['rotate'] = angle
        
    return balls

In [ ]:
shots: Shots = {'label': 'second serve', 'shots': []}

court_balls = lambda : driver.find_elements(By.XPATH, "//*[local-name()='g' and starts-with(@class, 'court-ball-')]")
    
passed_balls: set[str] = set()

for i in range(len(court_balls())):
    court_ball = court_balls()[i]
    
    cls = court_ball.get_attribute('class')
    if cls in passed_balls:
        continue
    
    shot: Shot = {}
    
    passed_balls.append(cls)
        
    child = court_ball.find_element(By.XPATH, r'./*')
    x, y = child.get_attribute('x'), child.get_attribute('y')
    x, y = float(x), float(y)
    x, y = to_real_pos(x, y, Court)
    
    
    
    court_ball.click()
    
    
    

    time.sleep(1)

    get_selected_group_elements().click()
    
    
    
    time.sleep(1)
    

In [ ]:
def get_balls():
    selected = driver.find_element(By.ID, "plottedBallsSelected")
    selected_balls = selected.find_elements(By.XPATH, r".//*[local-name()='g']/*")
    
    

In [ ]:
selected = driver.find_element(By.ID, "plottedBallsSelected")

In [ ]:
selected_balls = selected.find_elements(By.XPATH, r".//*[local-name()='g']/*")
[el.get_attribute('xlink:href') for el in selected_balls]


In [ ]:
selected_balls = selected.find_elements(By.XPATH, r".//*[local-name()='g']")

In [ ]:
ballEl = selected_balls[0]

In [ ]:
component = ballEl.find_elements(By.XPATH, r'./*')[0]

In [ ]:
component.get_attribute('x')

In [ ]:
extract_first_number(component.get_attribute('transform'))

In [ ]:
height, width = driver.find_element(By.XPATH, '//*[@id="CourtDoublesAlley"]').size.values()

In [ ]:
height/ width

In [ ]:
Court.heightM/ Court.widthM

In [ ]:
to_real_pos(0, 0, width, height, Court)

In [ ]:
width, height

In [ ]:
to_real_pos(-460, 96.68677287884401, width, height, Court)

In [ ]:
def get_curret_score_from_pop_up():
    game_scores = driver.find_element(By.CLASS_NAME, 'game-scores').text.split('\n')
    set_score_1 = tuple(map(int, driver.find_element(By.CLASS_NAME, 'set-scores-1').text.split('\n')))

    try:
        set_score_2 = tuple(map(int, driver.find_element(By.CLASS_NAME, 'set-scores-2').text.split('\n')))
    except:
        set_score_2 = None, None
        
    currentScore: CurrentScore =  {
        'player1': {
            'game_score': game_scores[0],
            'set_score_1':set_score_1[0],
            'set_score_2':set_score_2[0]
        },
        'player2': {
            'game_score': game_scores[1],
            'set_score_1':set_score_1[1],
            'set_score_2':set_score_2[1]
        }
    }

    return currentScore

In [ ]:
def get_current_shot(x: float, y: float):
    shot_description = driver.find_element(By.CLASS_NAME, 'shot-description').text
    player, typ = shot_description.lower().replace(' ','').split("'s")

    shot_description, player, typ

    shot: Shot =  {'shot_description': shot_description, 'player': player, 'type': typ}
    
    
    # get all attributes
    attributes = {}

    elements = driver.find_elements(By.XPATH, "//div[@class='header' and normalize-space(text())]")
    for element in elements:
        key = to_camel_case(element.text)
        value = sibling_of_div_text(element.text).text
        
        if value == 'NA':
            
            value = None
        attributes[key] = value


    shot['attributes'] = attributes
    shot['currentScore'] = get_curret_score_from_pop_up()
    shot['balls'] = []
    
    shot['position'] = {'x' : x, 'y': y}
    
    return shot
    

In [ ]:
get_current_shot()

In [ ]:
player1, player2 = (el.text for el in driver.find_elements(By.XPATH, r"//div[@class='player-info']/div[@class='name']/a"))
player1, player2

In [ ]:
import re

def extract_numbers_from_str(s: str, /):
    # Use regular expression to find all numbers inside brackets
    numbers = re.findall(r'\((\d+)\)', s)
    # Convert the numbers from strings to integers and return as a tuple
    return tuple(map(int, numbers))


### Stroke Summary

In [ ]:
from typing import Mapping


class StrokeSummary(TypedDict):
    strokes: list[Stroke]
    
class Stroke(TypedDict):
    strokeLabel: str
    player1: Hands
    player2: Hands
    

class Hand(TypedDict):
    backhand: int
    forehand: int    

class Hands(Mapping[str, Hand]):
    winners: Hand
    forcingShots: Hand
    unforcedErrors: Hand
    ralliesContinued: Hand





In [ ]:
data: StrokeSummary = {'strokes': []}

stats_wrapper = driver.find_element(By.CLASS_NAME, r"stats-wrapper")
strokes = stats_wrapper.find_elements(By.XPATH, './*')
len(strokes)

In [ ]:
if 'expand all' in driver.find_element(By.CLASS_NAME, 'expand-all').text.lower():
    driver.find_element(By.CLASS_NAME, 'expand-all').click()

In [ ]:
handsLabes = [
    "winners",
    "forcingShots",
    "unforcedErrors",
    "ralliesContinued"
]


In [ ]:
for stroke in strokes:
    strokeData: Stroke = {}
    strokeData["strokeLabel"] = stroke.find_element(By.XPATH, ".//div[@class='stroke-label-us']").text
    
    
    for player, i in (('player1', 1), ('player2', 3)):
        handsData: Hands = {}

        for hand, shot_label in zip(stroke.find_elements(By.XPATH, rf"./div/div[1]/div[1]/div[2]/div/div[3]/div[{i}]/div/*"), handsLabes):
            backhand, forehand = map(int, ' '.join(el.text for el in hand.find_elements(By.XPATH, './span')).split())            
            handsData[shot_label] = {"backhand": backhand, "forehand" : forehand}
            

        strokeData[player] = handsData
    data['strokes'].append(strokeData)




In [ ]:
from typing import Literal


handsData: Hands = {}

for hand, shot_label in zip(stroke.find_elements(By.XPATH, r"./div/div[1]/div[1]/div[2]/div/div[3]/div[1]/div/*"), handsLabes):
    backhand, forehand = map(int, ' '.join(el.text for el in hand.find_elements(By.XPATH, './span')).split())
    
    print(f"{backhand=} {forehand=}", shot_label)
    
    handsData[shot_label] = {"backhand": backhand, "forehand" : forehand}
    

handsData

In [ ]:
# labels = stroke.find_elements(By.XPATH, r"./div/div[1]/div[1]/div[2]/div/div[3]/div[2]/*")
# labels = [_lbl.text for _lbl in labels]
# labels

In [ ]:
for hand in stroke.find_elements(By.XPATH, r"./div/div[1]/div[1]/div[2]/div/div[3]/div[3]/div/*"):
    backhand, forehand = map(int, ' '.join(el.text for el in hand.find_elements(By.XPATH, './span')).split())
    print(f"{backhand=} {forehand=}")

In [ ]:
Stroke()['player1']['forcingShots']['forehand']

### Rally Analysis

In [ ]:
try:
    driver.find_element(By.XPATH, r"//button[text()='Expand All (X+E)']").click()
except NoSuchElementException:
    pass

In [ ]:
class RallyAnalysis(TypedDict):
    shortRally: list[Shot]
    mediumRally: list[Shot]
    longRally: list[Shot]
    
class Shot(TypedDict):
    i: int
    label: str
    player1: Point
    player2: Point
        
class Point(TypedDict):
    type: str
    count: int

In [ ]:
rallies = driver.find_element(By.CLASS_NAME, 'rallies')

shortRally = driver.find_element(By.ID, "shortRally")
mediumRally = driver.find_element(By.ID, "mediumRally")
longRally = driver.find_element(By.ID, "longRally")

rallies = [shortRally, mediumRally, longRally]
ralliesLabels = ['shortRally', 'mediumRally', 'longRally']


In [ ]:
data: RallyAnalysis = {}

for rally, rally_label in zip(rallies, ralliesLabels):
    shots = rally.find_elements(By.XPATH, r"./div/div[4]/div[2]/div[2]/*")
    data[rally_label] = []
    
    for i, shot in enumerate(shots):
        shotData: Shot = {}
        
        player1, shot_label, player2 = shot.find_elements(By.XPATH, './*')
        
        shotData['i'] = i
        shotData['label'] = shot_label.find_element(By.XPATH, r"./div").text
        
        for player, playerLabel in ((player1, 'player1'), (player2, 'player2')):
            shotData[playerLabel] = []    
            
            points = player.find_elements(By.XPATH,'./div')
            for point in points:
                count, type = point.find_element(By.XPATH, r'./div/div').text.split('\n')
                point: Point = {'count': int(count), 'type': type}
                shotData[playerLabel].append(point)
        
        data[rally_label].append(shotData)
with open('rally_analysis.json', mode='w') as file:
    json.dump(data, file, indent=4)


In [ ]:
shot = shots[0]

In [ ]:
player1, shot_label, player2 = shot.find_elements(By.XPATH, './*')

In [ ]:
pointsEl = player1.find_elements(By.XPATH,'./div')

In [ ]:
for pointEl in pointsEl:
    print(pointEl.find_element(By.XPATH, r'./div/div').text)
    print()